In [28]:
import pandas as pd
from scipy import stats
import numpy as np

# Load the dataset
df = pd.read_csv('AB_NYC_2019.csv')

# 1. Data Integrity

# Convert `last_review` to datetime
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

In [29]:
# Convert numeric columns to appropriate types
numeric_columns = [
    'id', 'host_id', 'latitude', 'longitude', 'price', 
    'minimum_nights', 'number_of_reviews', 'reviews_per_month', 
    'calculated_host_listings_count', 'availability_365'
]
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Verify categorical fields contain valid and expected values
# Convert categorical fields to lowercase and strip any extra spaces
categorical_columns = [
    'name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'room_type'
]
df[categorical_columns] = df[categorical_columns].apply(lambda x: x.str.lower().str.strip())

# Check for unique values in categorical fields
for col in categorical_columns:
    print(f"Unique values in '{col}':\n", df[col].unique())


Unique values in 'name':
 ['clean & quiet apt home by the park' 'skylit midtown castle'
 'the village of harlem....new york !' ...
 'sunny studio at historical neighborhood'
 '43rd st. time square-cozy single bed'
 "trendy duplex in the very heart of hell's kitchen"]
Unique values in 'host_name':
 ['john' 'jennifer' 'elisabeth' ... 'abayomi' 'alberth' 'ilgar & aysel']
Unique values in 'neighbourhood_group':
 ['brooklyn' 'manhattan' 'queens' 'staten island' 'bronx']
Unique values in 'neighbourhood':
 ['kensington' 'midtown' 'harlem' 'clinton hill' 'east harlem'
 'murray hill' 'bedford-stuyvesant' "hell's kitchen" 'upper west side'
 'chinatown' 'south slope' 'west village' 'williamsburg' 'fort greene'
 'chelsea' 'crown heights' 'park slope' 'windsor terrace' 'inwood'
 'east village' 'greenpoint' 'bushwick' 'flatbush' 'lower east side'
 'prospect-lefferts gardens' 'long island city' 'kips bay' 'soho'
 'upper east side' 'prospect heights' 'washington heights' 'woodside'
 'brooklyn heights'

In [30]:
# 2. Missing Data Handling

# Check for missing values
missing_values = df.isnull().sum()
print("Missing values before handling:\n", missing_values)

Missing values before handling:
 id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64


In [31]:
# Define strategies for handling missing values
# Fill missing values in `reviews_per_month` with 0 (assuming no reviews means 0 reviews per month)
df['reviews_per_month'].fillna(0, inplace=True)

C:\Users\ANKIT\AppData\Local\Temp\ipykernel_10720\806112462.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['reviews_per_month'].fillna(0, inplace=True)


In [32]:
# Fill missing values in `last_review` with the earliest review date in the dataset
# If 'last_review' column has missing values, fill them with the earliest review date
if df['last_review'].isnull().any():
    earliest_review_date = df['last_review'].min()
    df['last_review'].fillna(earliest_review_date, inplace=True)

C:\Users\ANKIT\AppData\Local\Temp\ipykernel_10720\3883386831.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['last_review'].fillna(earliest_review_date, inplace=True)


In [33]:
# Remove rows with missing values in critical fields
df.dropna(subset=['name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price'], inplace=True)

In [34]:
# Check for missing values after handling
missing_values_after = df.isnull().sum()
print("Missing values after handling:\n", missing_values_after)

Missing values after handling:
 id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64


In [35]:
# 3. Duplicate Removal

# Check for duplicates
duplicate_rows = df.duplicated()
print(f"Number of duplicate rows: {duplicate_rows.sum()}")

Number of duplicate rows: 0


In [36]:
# Remove duplicates
df_cleaned = df.drop_duplicates()

In [37]:
# Verify that duplicates have been removed
duplicate_rows_after = df_cleaned.duplicated()
print(f"Number of duplicate rows after removal: {duplicate_rows_after.sum()}")

Number of duplicate rows after removal: 0


In [38]:
# 4. Standardization

# Standardize text fields: Convert to lowercase and strip extra spaces
text_fields = ['name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'room_type']
df[text_fields] = df[text_fields].apply(lambda x: x.str.lower().str.strip())

In [39]:
# Ensure numeric fields are within realistic ranges
# Example checks for latitude and longitude ranges
df = df[(df['latitude'] >= -90) & (df['latitude'] <= 90)]
df = df[(df['longitude'] >= -180) & (df['longitude'] <= 180)]

In [40]:
# Ensure prices are positive and reasonable (assuming a max price of 10,000 for this example)
df = df[df['price'] > 0]
df = df[df['price'] <= 10000]

In [41]:
# Ensure `minimum_nights` is positive
df = df[df['minimum_nights'] > 0]

In [43]:
# Standardize date formats
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

In [60]:
# 5. Outlier Detection

# Function to detect outliers using Z-score method
def detect_outliers_zscore(data, threshold=3):
    z_scores = np.abs(stats.zscore(data))
    return np.where(z_scores > threshold)

In [56]:
# Detect outliers in numeric columns and address them
numeric_columns = ['price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']

In [58]:
# Initialize a DataFrame to store the indices of outliers
outliers_indices = pd.DataFrame()

for column in numeric_columns:
    outliers = detect_outliers_zscore(df[column].dropna())
    outliers_indices[column] = pd.Series(outliers[0])

In [59]:
# Verify if outliers are removed
for column in numeric_columns:
    print(f"Outliers removed in '{column}': {outliers_indices[column].dropna().size}")

Outliers removed in 'price': 388
Outliers removed in 'minimum_nights': 327
Outliers removed in 'number_of_reviews': 388
Outliers removed in 'reviews_per_month': 388
Outliers removed in 'calculated_host_listings_count': 388
Outliers removed in 'availability_365': 0
